In [29]:
install.packages(c("car","broom","emmeans"))

also installing the dependencies ‘estimability’, ‘mvtnorm’





The downloaded binary packages are in
	/var/folders/d4/7g51j6qd0zn5cpcjtsbx5npc0000gp/T//RtmpmX2Ooc/downloaded_packages


In [1]:
source("process/process.R")


********************* PROCESS for R Version 4.3.1 ********************* 
 
           Written by Andrew F. Hayes, Ph.D.  www.afhayes.com              
   Documentation available in Hayes (2022). www.guilford.com/p/hayes3   
 
*********************************************************************** 
 
PROCESS is now ready for use.
Copyright 2020-2023 by Andrew F. Hayes ALL RIGHTS RESERVED
Workshop schedule at http://haskayne.ucalgary.ca/CCRAM
 


In [2]:
raw_df <- read.csv("../results/pilot_final_survey3_normal.csv", check.names = FALSE)

In [ ]:
short_col_df <- raw_df
names(short_col_df) <- substr(names(raw_df), 1, 3)

# Print the resulting data frame
short_col_df

In [9]:
df <- short_col_df[-1, ]

to_number_if_number <- function(s) {
  if (grepl("^[0-9]+$", s)) {
    as.integer(s)
  } else {
    s
  }
}

for (i in 1:55) {
  col_name <- sprintf("%03d", i)
  df[[col_name]] <- sapply(df[[col_name]], function(x) {
    val <- sub("^([a-zA-Z0-9]+).*", "\\1", x)
    to_number_if_number(val)
  })
}

df

,Tri,Sce,001,002,003,004,005,006,007,008,⋯,046,047,048,049,050,051,052,053,054,055
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,⋯,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<int>
2,0,"Disadvantaged customer, based on past browsing behavior",b,62,e,a,f,4,2,6,⋯,6,5,c,d,g,a,a,a,6,4
3,1,"Favored customer, based on device used",b,67,e,c,d,5,4,5,⋯,5,4,c,f,e,c,b,b,6,5
4,10,"Disadvantaged customer, based on past browsing behavior",b,62,d,f,d,4,5,2,⋯,6,6,c,e,g,a,b,a,6,6
5,11,"Disadvantaged customer, based on past browsing behavior",a,59,d,a,d,5,6,4,⋯,6,6,c,c,e,b,b,a,6,4
6,12,"Favored customer, based on past browsing behavior",a,32,e,f,f,5,4,5,⋯,5,4,c,f,f,a,c,a,6,6
7,13,"Favored customer, based on device used",b,62,b,a,d,5,6,4,⋯,5,4,c,c,e,a,b,b,6,7
8,14,"Favored customer, based on past browsing behavior",b,53,d,a,d,4,5,6,⋯,4,3,c,f,d,a,c,a,6,6
9,15,"Disadvantaged customer, based on geographical location",b,32,e,a,e,5,4,5,⋯,6,4,c,f,e,a,b,c,6,6
10,16,"Favored customer, based on device used",b,52,e,a,d,5,2,4,⋯,6,4,c,d,g,a,b,b,6,6


In [10]:
first_row <- as.data.frame(t(short_col_df[1, , drop = FALSE]), stringsAsFactors = FALSE)
colnames(first_row) <- "groups"

column_group_index <- function(group_name) {
  matching <- first_row$groups == group_name
  cols = rownames(first_row)[matching]
  valid_cols <- intersect(cols, names(df))
  return(valid_cols)
}

In [17]:
df$price_fairness <- rowMeans(sapply(column_group_index("01 Distributive Price Fairness Perception"), function(col) as.integer(df[[col]])))
df$purchase_intention <- rowMeans(sapply(column_group_index("03 Purchase Intention"), function(col) as.integer(df[[col]])))
df$future_search_intention <- rowMeans(sapply(column_group_index("04 Future Search Intention"), function(col) as.integer(df[[col]])))
df$product_involvement <- rowMeans(sapply(column_group_index("08 Product Involvement"), function(col) as.integer(df[[col]])))
df$benevolence_trust <- rowMeans(sapply(column_group_index("05 Benevolence Trust"), function(col) as.integer(df[[col]])))
df$private_complaint_intention <- rowMeans(sapply(column_group_index("06 Private Complaint Intentions"), function(col) as.integer(df[[col]])))
df$public_complaint_intention <- rowMeans(sapply(column_group_index("07 Public Complaint Intentions"), function(col) as.integer(df[[col]])))
df$perceived_data_sensitivity <- rowMeans(sapply(column_group_index("10 Perceived Data Sensitivity"), function(col) as.integer(df[[col]])))
df$internet_privacy_concerns <- rowMeans(sapply(column_group_index("09 Internet Privacy Concern"), function(col) as.integer(df[[col]])))

df

,Tri,Sce,001,002,003,004,005,006,007,008,⋯,education,employment,social_media_usage,internet_usage,money_spent,benevolence_trust,private_complaint_intention,public_complaint_intention,perceived_data_sensitivity,internet_privacy_concerns
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,0,"Disadvantaged customer, based on past browsing behavior",b,62,e,a,f,4,2,6,⋯,e,a,a,c,g,2.8,5.50,3.0,5.2,4.75
3,1,"Favored customer, based on device used",b,67,e,c,d,5,4,5,⋯,e,c,c,c,e,3.2,5.00,4.0,4.2,6.00
4,10,"Disadvantaged customer, based on past browsing behavior",b,62,d,f,d,4,5,2,⋯,d,f,a,c,g,3.2,5.00,4.4,5.4,6.25
5,11,"Disadvantaged customer, based on past browsing behavior",a,59,d,a,d,5,6,4,⋯,d,a,b,c,e,4.0,5.25,3.6,5.8,5.00
6,12,"Favored customer, based on past browsing behavior",a,32,e,f,f,5,4,5,⋯,e,f,a,c,f,4.8,4.50,2.8,4.2,5.25
7,13,"Favored customer, based on device used",b,62,b,a,d,5,6,4,⋯,b,a,a,c,e,4.2,5.25,3.8,4.6,5.75
8,14,"Favored customer, based on past browsing behavior",b,53,d,a,d,4,5,6,⋯,d,a,a,c,d,2.6,5.25,3.2,4.2,5.25
9,15,"Disadvantaged customer, based on geographical location",b,32,e,a,e,5,4,5,⋯,e,a,a,c,e,4.4,5.25,3.4,4.8,5.75
10,16,"Favored customer, based on device used",b,52,e,a,d,5,2,4,⋯,e,a,a,c,g,2.6,5.00,3.4,4.4,6.00


In [18]:
df$age <- df[["002"]]
df$gender <- df[["001"]]
df$dummy_gender <- as.integer(df$gender == "b")
df$income <- df[["005"]]
df$dummy_income <- as.integer(df$income %in% c("d", "e", "f"))
df$frequency <- df[[column_group_index("12 Frequency of Online Purchases")[1]]]
df$dummy_frequency <- as.integer(df$frequency == "f")

df$education = df[["003"]]
df$employment = df[["004"]]
df$social_media_usage = df[[column_group_index("14 Social Media Usage")[1]]]

df$internet_usage = df[[column_group_index("11 Daily Internet Usage")[1]]]
df$money_spent = df[[column_group_index("13 Money Spent for Online Purchases")[1]]]



df

,Tri,Sce,001,002,003,004,005,006,007,008,⋯,education,employment,social_media_usage,internet_usage,money_spent,benevolence_trust,private_complaint_intention,public_complaint_intention,perceived_data_sensitivity,internet_privacy_concerns
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,0,"Disadvantaged customer, based on past browsing behavior",b,62,e,a,f,4,2,6,⋯,e,a,a,c,g,2.8,5.50,3.0,5.2,4.75
3,1,"Favored customer, based on device used",b,67,e,c,d,5,4,5,⋯,e,c,c,c,e,3.2,5.00,4.0,4.2,6.00
4,10,"Disadvantaged customer, based on past browsing behavior",b,62,d,f,d,4,5,2,⋯,d,f,a,c,g,3.2,5.00,4.4,5.4,6.25
5,11,"Disadvantaged customer, based on past browsing behavior",a,59,d,a,d,5,6,4,⋯,d,a,b,c,e,4.0,5.25,3.6,5.8,5.00
6,12,"Favored customer, based on past browsing behavior",a,32,e,f,f,5,4,5,⋯,e,f,a,c,f,4.8,4.50,2.8,4.2,5.25
7,13,"Favored customer, based on device used",b,62,b,a,d,5,6,4,⋯,b,a,a,c,e,4.2,5.25,3.8,4.6,5.75
8,14,"Favored customer, based on past browsing behavior",b,53,d,a,d,4,5,6,⋯,d,a,a,c,d,2.6,5.25,3.2,4.2,5.25
9,15,"Disadvantaged customer, based on geographical location",b,32,e,a,e,5,4,5,⋯,e,a,a,c,e,4.4,5.25,3.4,4.8,5.75
10,16,"Favored customer, based on device used",b,52,e,a,d,5,2,4,⋯,e,a,a,c,g,2.6,5.00,3.4,4.4,6.00


In [19]:
vars <- c("gender", "education", "employment")

for (i in 1:length(vars)) { 
    for (j in 1:i) {
        if (i != j){
            cat("Chi-squared test for", vars[i], "vs", vars[j])
            tab <- table(df[[vars[i]]], df[[vars[j]]])
            print(tab)
            test <- chisq.test(tab)
            print(test)
        }
    }
}

Chi-squared test for education vs gender   
     a  b
  a  4  5
  b  6  3
  c 10  5
  d 13 21
  e 10 13
  f  8  2


Warning message in chisq.test(tab):
“Chi-squared approximation may be incorrect”



	Pearson's Chi-squared test

data:  tab
X-squared = 8.6149, df = 5, p-value = 0.1254

Chi-squared test for employment vs gender   
     a  b
  a 16 17
  b  4  1
  c 17 14
  d  4  4
  e  4  2
  f  5  7
  g  1  4


Warning message in chisq.test(tab):
“Chi-squared approximation may be incorrect”



	Pearson's Chi-squared test

data:  tab
X-squared = 4.8826, df = 6, p-value = 0.559

Chi-squared test for employment vs education   
     a  b  c  d  e  f
  a  1  1  4 13 10  4
  b  1  1  0  2  1  0
  c  4  2  6  9  4  6
  d  1  2  1  3  1  0
  e  1  1  1  2  1  0
  f  0  1  3  3  5  0
  g  1  1  0  2  1  0


Warning message in chisq.test(tab):
“Chi-squared approximation may be incorrect”



	Pearson's Chi-squared test

data:  tab
X-squared = 25.098, df = 30, p-value = 0.7202



In [20]:
vars <- c("age", "income")

for (var in vars) {
  cat("Kruskal-Wallis test for", var, "vs Sce\n")
  result <- kruskal.test(df[[var]] ~ df$Sce)
  print(result)
  cat("\n--------------------------------\n\n")
}

Kruskal-Wallis test for age vs Sce

	Kruskal-Wallis rank sum test

data:  df[[var]] by df$Sce
Kruskal-Wallis chi-squared = 8.9718, df = 5, p-value = 0.1102


--------------------------------

Kruskal-Wallis test for income vs Sce

	Kruskal-Wallis rank sum test

data:  df[[var]] by df$Sce
Kruskal-Wallis chi-squared = 6.0507, df = 5, p-value = 0.3013


--------------------------------



In [21]:
library(dplyr)

cat <- "17 Realism Check"
cols <- column_group_index(cat)

for (col in cols) {
  cat("ANOVA for:", col, "\n\n")
  
  # Descriptive statistics
  summary_df <- df %>%
    group_by(Sce) %>%
    summarize(
      M  = round(mean(.data[[col]], na.rm = TRUE), 2),
      SD = round(sd(.data[[col]], na.rm = TRUE), 2),
      n  = n()
    )
  print(summary_df)
  
  # One-way ANOVA
  fit <- aov(df[[col]] ~ Sce, data = df)
  print(summary(fit))
  
  cat("\n----------------------------------------\n\n")
}

ANOVA for: 054 

# A tibble: 6 × 4
  Sce                                                         M    SD     n
  <chr>                                                   <dbl> <dbl> <int>
1 Disadvantaged customer, based on device used             6     0       18
2 Disadvantaged customer, based on geographical location   6.09  0.54    11
3 Disadvantaged customer, based on past browsing behavior  5.85  0.49    20
4 Favored customer, based on device used                   5.85  0.59    20
5 Favored customer, based on geographical location         6     0.35    17
6 Favored customer, based on past browsing behavior        5.86  0.53    14
            Df Sum Sq Mean Sq F value Pr(>F)
Sce          5  0.787  0.1573    0.75  0.588
Residuals   94 19.723  0.2098               

----------------------------------------

ANOVA for: 055 

# A tibble: 6 × 4
  Sce                                                         M    SD     n
  <chr>                                                   <dbl> <dbl

In [22]:
variablen <- c(
    "price_fairness",
    "benevolence_trust",
    "purchase_intention", 
    "future_search_intention",
    "private_complaint_intention",
    "public_complaint_intention",
    "perceived_data_sensitivity",
    "internet_privacy_concerns",
    "product_involvement")

n <- length(variablen)

cor_matrix <- matrix("", nrow = n, ncol = n, dimnames = list(variablen, variablen))
p_values <- matrix(NA, nrow = n, ncol = n, dimnames = list(variablen, variablen))

for (i in seq_along(variablen)) {
  for (j in seq_along(variablen)) {
    if (i == j) {
      cor_matrix[i, j] <- "1.000"
    } else {
      test <- cor.test(df[[variablen[i]]], df[[variablen[j]]])
      r <- test$estimate
      p <- test$p.value
      stars <- if (p < 0.01) {
        "***"
      } else if (p < 0.05) {
        "**"
      } else if (p < 0.10) {
        "*"
      } else {
        ""
      }
      cor_matrix[i, j] <- sprintf("%.3f%s", r, stars)
      p_values[i, j] <- p
    }
  }
}

means <- round(colMeans(df[variablen], na.rm = TRUE), 2)
stds <- round(apply(df[variablen], 2, sd, na.rm = TRUE), 2)

cor_matrix <- rbind(cor_matrix, Mean = as.character(means), `Standard Deviation` = as.character(stds))
cor_matrix

,price_fairness,benevolence_trust,purchase_intention,future_search_intention,private_complaint_intention,public_complaint_intention,perceived_data_sensitivity,internet_privacy_concerns,product_involvement
price_fairness,1.000,0.038,0.216**,0.159,0.161,-0.034,-0.037,0.013,-0.049
benevolence_trust,0.038,1.000,0.311***,0.028,-0.140,-0.138,-0.138,-0.020,0.058
purchase_intention,0.216**,0.311***,1.000,-0.193*,-0.028,-0.123,-0.038,-0.039,-0.099
future_search_intention,0.159,0.028,-0.193*,1.000,0.217**,0.062,0.225**,0.234**,-0.008
private_complaint_intention,0.161,-0.140,-0.028,0.217**,1.000,0.322***,0.209**,0.181*,-0.025
public_complaint_intention,-0.034,-0.138,-0.123,0.062,0.322***,1.000,0.155,0.240**,0.046
perceived_data_sensitivity,-0.037,-0.138,-0.038,0.225**,0.209**,0.155,1.000,0.193*,-0.024
internet_privacy_concerns,0.013,-0.020,-0.039,0.234**,0.181*,0.240**,0.193*,1.000,0.094
product_involvement,-0.049,0.058,-0.099,-0.008,-0.025,0.046,-0.024,0.094,1.000
Mean,4.68,3.77,5.52,6,5.09,3.46,4.89,5.55,5.16


In [28]:
library(dplyr)
library(knitr)
library(tidyr)

columns <- c(
  "price_fairness",
  "purchase_intention", 
  "future_search_intention",
  "benevolence_trust",
  "private_complaint_intention",
  "public_complaint_intention"
)

summary_table <- df %>%
  group_by(Sce) %>%
  summarise(across(all_of(columns),
                   list(Mean = ~mean(. , na.rm = TRUE),
                        SD   = ~sd(., na.rm = TRUE)),
                   .names = "{col}_{fn}"))

summary_table

Sce,price_fairness_Mean,price_fairness_SD,purchase_intention_Mean,purchase_intention_SD,future_search_intention_Mean,future_search_intention_SD,benevolence_trust_Mean,benevolence_trust_SD,private_complaint_intention_Mean,private_complaint_intention_SD,public_complaint_intention_Mean,public_complaint_intention_SD
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Disadvantaged customer, based on device used",4.648148,0.4349476,5.444444,0.5239368,6.037037,0.3772776,3.800000,0.6686246,5.180556,0.3186086,3.555556,0.6080076
"Disadvantaged customer, based on geographical location",4.939394,0.1348400,5.575758,0.2620818,6.212121,0.3423035,3.854545,0.6817091,5.113636,0.3599242,3.218182,0.6539391
"Disadvantaged customer, based on past browsing behavior",4.550000,0.4988290,5.583333,0.4171050,5.866667,0.3958853,3.840000,0.6946411,5.100000,0.4397966,3.340000,0.7111148
"Favored customer, based on device used",4.666667,0.4588315,5.416667,0.5058141,6.016667,0.3969916,3.410000,0.5210314,5.137500,0.4622243,3.560000,0.6210348
"Favored customer, based on geographical location",4.588235,0.4171566,5.686275,0.1852396,6.137255,0.4258786,4.082353,0.5341293,4.926471,0.5712152,3.435294,0.6451174
"Favored customer, based on past browsing behavior",4.833333,0.4082483,5.428571,0.6328415,5.785714,0.5165160,3.714286,0.9003052,5.053571,0.4404824,3.571429,0.9506504


In [34]:
library(dplyr)
library(tidyr)

df <- df %>%
  # split at the “, based on ”
  separate(Sce,
           into = c("price_change_raw", "tactic_raw"),
           sep   = ", based on ",
           remove = FALSE) %>%
  # recode to nice short labels
  mutate(
    price_change = case_when(
      price_change_raw == "Favored customer"       ~ "favored",
      price_change_raw == "Disadvantaged customer" ~ "disadvantaged",
      TRUE                                         ~ NA_character_
    ),
    tactic = case_when(
      tactic_raw == "device used"                ~ "device",
      tactic_raw == "geographical location"      ~ "geographical_location",
      tactic_raw == "past browsing behavior"     ~ "browsing_behavior",
      TRUE                                       ~ NA_character_
    )
  ) %>%
  # drop the raws if you like
  select(-price_change_raw, -tactic_raw) %>%
  # make them factors in the desired order
  mutate(
    price_change = factor(price_change, levels = c("favored","disadvantaged")),
    tactic       = factor(tactic,       levels = c("device",
                                                   "geographical_location",
                                                   "browsing_behavior"))
  )
df

,Tri,Sce,001,002,003,004,005,006,007,008,⋯,social_media_usage,internet_usage,money_spent,benevolence_trust,private_complaint_intention,public_complaint_intention,perceived_data_sensitivity,internet_privacy_concerns,price_change,tactic
,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<int>,⋯,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
2,0,"Disadvantaged customer, based on past browsing behavior",b,62,e,a,f,4,2,6,⋯,a,c,g,2.8,5.50,3.0,5.2,4.75,disadvantaged,browsing_behavior
3,1,"Favored customer, based on device used",b,67,e,c,d,5,4,5,⋯,c,c,e,3.2,5.00,4.0,4.2,6.00,favored,device
4,10,"Disadvantaged customer, based on past browsing behavior",b,62,d,f,d,4,5,2,⋯,a,c,g,3.2,5.00,4.4,5.4,6.25,disadvantaged,browsing_behavior
5,11,"Disadvantaged customer, based on past browsing behavior",a,59,d,a,d,5,6,4,⋯,b,c,e,4.0,5.25,3.6,5.8,5.00,disadvantaged,browsing_behavior
6,12,"Favored customer, based on past browsing behavior",a,32,e,f,f,5,4,5,⋯,a,c,f,4.8,4.50,2.8,4.2,5.25,favored,browsing_behavior
7,13,"Favored customer, based on device used",b,62,b,a,d,5,6,4,⋯,a,c,e,4.2,5.25,3.8,4.6,5.75,favored,device
8,14,"Favored customer, based on past browsing behavior",b,53,d,a,d,4,5,6,⋯,a,c,d,2.6,5.25,3.2,4.2,5.25,favored,browsing_behavior
9,15,"Disadvantaged customer, based on geographical location",b,32,e,a,e,5,4,5,⋯,a,c,e,4.4,5.25,3.4,4.8,5.75,disadvantaged,geographical_location
10,16,"Favored customer, based on device used",b,52,e,a,d,5,2,4,⋯,a,c,g,2.6,5.00,3.4,4.4,6.00,favored,device


In [35]:
library(car)
library(broom)
library(emmeans)

DVs    <- c("price_fairness", "purchase_intention", 
            "future_search_intention", "benevolence_trust",
            "private_complaint_intention", "public_complaint_intention")

covars <- c("perceived_data_sensitivity", "internet_privacy_concerns", "product_involvement",
            "age", "gender", "income", "frequency", "employment")

results_list <- list()

for (dv in DVs) {
  
  cat("\n\n### Dependent Variable:", dv, "\n")
  
  # 3a. Levene’s test for homogeneity of variance
  lt <- leveneTest(
    y       = as.formula(paste(dv, "~ price_change * tactic")),
    data    = df
  )
  print(lt)
  
  # 3b. Fit full ANCOVA model (Type III SS)
  formula_text <- paste(
    dv,
    "~ price_change * tactic +",
    paste(covars, collapse = " + ")
  )
  mod <- lm(as.formula(formula_text), data = df)
  
  # 3c. Normality of residuals
  sh  <- shapiro.test(resid(mod))
  cat("Shapiro-Wilk p-value:", round(sh$p.value, 3), "\n")
  
  # 3d. Type III ANCOVA table
  ancova_tbl <- Anova(mod, type = "III")
  print(ancova_tbl)
  
  # 3e. Effect sizes (partial eta²)
  eta2_tbl <- with(ancova_tbl, 
                   data.frame(
                     Effect = rownames(ancova_tbl),
                     F      = `F value`,
                     p      = `Pr(>F)`,
                     eta2  = `Sum Sq` / (`Sum Sq` + ancova_tbl["Residuals","Sum Sq"])
                   ))
  print(eta2_tbl)
  
  # 3f. Estimated marginal means for the 2×3 interaction
  emm <- emmeans(mod, ~ price_change * tactic)
  print(summary(emm, infer = TRUE))
  
  # store for later if desired
  results_list[[dv]] <- list(
    model   = mod,
    ancova  = ancova_tbl,
    eta2    = eta2_tbl,
    emmeans = emm
  )
}



### Dependent Variable: price_fairness 
Levene's Test for Homogeneity of Variance (center = median)
      Df F value  Pr(>F)  
group  5  2.2672 0.05399 .
      94                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
Shapiro-Wilk p-value: 0.01 
Anova Table (Type III tests)

Response: price_fairness
                            Sum Sq Df F value    Pr(>F)    
(Intercept)                 7.3385  1 37.5379 3.969e-08 ***
price_change                0.1064  1  0.5444   0.46295    
tactic                      0.6161  2  1.5758   0.21371    
perceived_data_sensitivity  0.0011  1  0.0055   0.94084    
internet_privacy_concerns   0.0147  1  0.0753   0.78455    
product_involvement         0.3161  1  1.6167   0.20754    
age                         0.1042  1  0.5331   0.46761    
gender                      0.4911  1  2.5120   0.11725    
income                      0.4773  4  0.6103   0.65647    
frequency                   0.7649  5  0.7825   0.56550    
employm